<a href="https://colab.research.google.com/github/gauravm999/LLMs/blob/main/Optimize_Retail_Store_visits_per_Salesman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimize Number of Store visits by Salesman
 The goal is to identify  the number of visits per store (by a salesman) that will maximize sales. This is done by predicting how sales vary when the visit frequency changes and selecting the frequency that results in the highest sales.
 https://chatgpt.com/c/66e5d1e2-8e4c-800a-9638-bdb72d9a8e92

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb

# Sample dataset creation
data = {
    'Store_ID': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'Visit_Frequency': [1, 2, 3, 1, 2, 3, 1, 2, 3],  # Visits per week
    'Store_Size': ['small', 'small', 'small', 'medium', 'medium', 'medium', 'large', 'large', 'large'],  # Categorical
    'Stock_Level': [50, 100, 150, 80, 160, 240, 100, 200, 300],  # Inventory level
    'Promotion_Active': [0, 1, 1, 0, 0, 1, 0, 0, 1],  # 0 = No, 1 = Yes
    'Sales': [400, 600, 800, 500, 700, 900, 600, 800, 1000]  # Target variable
}

df = pd.DataFrame(data)
print(df)

   Store_ID  Visit_Frequency Store_Size  Stock_Level  Promotion_Active  Sales
0         1                1      small           50                 0    400
1         1                2      small          100                 1    600
2         1                3      small          150                 1    800
3         2                1     medium           80                 0    500
4         2                2     medium          160                 0    700
5         2                3     medium          240                 1    900
6         3                1      large          100                 0    600
7         3                2      large          200                 0    800
8         3                3      large          300                 1   1000


In [ ]:
# One-hot encode Store_Size and Promotion_Active
df_encoded = pd.get_dummies(df, columns=['Store_Size', 'Promotion_Active'], drop_first=True)
print(df_encoded)

   Store_ID  Visit_Frequency  Stock_Level  Sales  Store_Size_medium  \
0         1                1           50    400              False   
1         1                2          100    600              False   
2         1                3          150    800              False   
3         2                1           80    500               True   
4         2                2          160    700               True   
5         2                3          240    900               True   
6         3                1          100    600              False   
7         3                2          200    800              False   
8         3                3          300   1000              False   

   Store_Size_small  Promotion_Active_1  
0              True               False  
1              True                True  
2              True                True  
3             False               False  
4             False               False  
5             False                Tr

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

# One-hot encode Store_Size and Promotion_Active
df_encoded = pd.get_dummies(df, columns=['Store_Size', 'Promotion_Active'], drop_first=True)

# Features and Target
X = df_encoded.drop(['Sales'], axis=1)
y = df_encoded['Sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to DMatrix for XGBoost
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# Features and Target
X = df_encoded.drop(['Sales'], axis=1)
y = df_encoded['Sales']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to DMatrix for XGBoost
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # Regression
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Train the XGBoost model
model = xgb.train(params, train_data, num_boost_round=100)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:59:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Function to find optimal visit frequency for a store
def find_optimal_visits(store_id, model, df_encoded):
    store_data = df_encoded[df_encoded['Store_ID'] == store_id].copy()

    visit_freq_range = [1, 2, 3, 4, 5]  # Vary the visit frequency
    sales_predictions = []

    for freq in visit_freq_range:
        # Modify visit frequency for the store
        store_data['Visit_Frequency'] = freq

        # Predict sales using the trained model
        dmatrix_store = xgb.DMatrix(store_data.drop(['Sales'], axis=1))
        predicted_sales = model.predict(dmatrix_store)
        sales_predictions.append(predicted_sales.mean())

    # Find the optimal number of visits that maximizes predicted sales
    optimal_visits = visit_freq_range[sales_predictions.index(max(sales_predictions))]

    return optimal_visits, sales_predictions

In [ ]:
# Find optimal number of visits for Store 1
optimal_visits, sales_preds = find_optimal_visits(1, model, df_encoded)
print(f"Optimal visits for Store 1: {optimal_visits}")
print(f"Predicted sales for each visit frequency: {sales_preds}")

Optimal visits for Store 1: 3
Predicted sales for each visit frequency: [420.084, 701.7067, 789.5892, 789.5892, 789.5892]


In [ ]:
# Find optimal number of visits for all stores
unique_stores = df_encoded['Store_ID'].unique()

optimal_visits_results = {}

for store in unique_stores:
    optimal_visits, sales_preds = find_optimal_visits(store, model, df_encoded)
    optimal_visits_results[store] = {
        'Optimal Visits': optimal_visits,
        'Predicted Sales': sales_preds
    }

# Print results for all stores
for store, results in optimal_visits_results.items():
    print(f"Optimal visits for Store {store}: {results['Optimal Visits']}")
    print(f"Predicted sales for Store {store} for each visit frequency: {results['Predicted Sales']}")
    print("-" * 50)

Optimal visits for Store 1: 3
Predicted sales for Store 1 for each visit frequency: [420.084, 701.7067, 789.5892, 789.5892, 789.5892]
--------------------------------------------------
Optimal visits for Store 2: 3
Predicted sales for Store 2 for each visit frequency: [505.673, 718.9815, 853.867, 853.867, 853.867]
--------------------------------------------------
Optimal visits for Store 3: 3
Predicted sales for Store 3 for each visit frequency: [609.5933, 816.7205, 941.98956, 941.98956, 941.98956]
--------------------------------------------------
